In [18]:
import seaborn as sns
import re
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import math
import numpy as np
%matplotlib inline

## Read data, convert all Excel non-string types to Python strings

In [19]:
converters = {'TimeFinished': str,' On a scale of 1 to 10, how reasonable do you think the public reaction is to COVID-19 now? (1 is under-reacting and 10 is overreacting)':str,'On a scale of 1 to 10, how much do you trust the government to respond to COVID-19 effectively? (1 is strongly distrust and 10 is strongly trust)':str,' Over the past week, have you been continually worried or anxious about a number of events or activities in your daily life?':str,'What is the zip code (i.e. postcal code) of the place you currently live in?':str}
data = pd.read_excel('Master.xlsx',sheet_name='Master',converters=converters)

## Strip spaces from beginning of strings, standardize "No Response" as NaN

In [20]:
for col in data.columns:
    if data[col].dtype == 'O':
        data[col] = data[col].str.lstrip()

data = data.replace('NR', np.NaN)
data = data.replace('Invalid', np.NaN)

## Standardize first letter capitalization

In [21]:
data['What is your employment status?'] = data['What is your employment status?'].str.capitalize() 
data['What is your gender?'] = data['What is your gender?'].str.capitalize()
data['What is your ethnicity?'] = data['What is your ethnicity?'].str.capitalize() 

## Create Datetime, Day, Age, Age Group variables

In [22]:
data['Datetime'] = pd.to_datetime(data.TimeFinished)
data['Day'] = data['Datetime'].dt.strftime("%b-%d")
data['Age'] = 2020 - data['What year were you born?']
data['Age Groups'] = pd.cut(x=data['Age'], bins=[0, 24, 39, 54, 70, 100]).apply(str)

## Create Dummy Variables, (Female,Male,Student,Retired)

In [23]:
data[['Female','Male']] = pd.get_dummies(data['What is your gender?'])[['Female','Male']]
data[['Student','Retired']] = pd.get_dummies(data['What is your employment status?'])[['Student','Retired']]


## WFH? column cleaning

In [24]:
data['In the past 3 days, did you work from home (WFH)?'] = data['In the past 3 days, did you work from home (WFH)?'].str.replace('home on some days','home for some days')
data['In the past 3 days, did you work from home (WFH)?'] = data['In the past 3 days, did you work from home (WFH)?'].str.replace('provide an option for me to work from home','provide an option to WFH')

## Income column cleaning

In [25]:
incomelist = data['What is your annual household income?']
incomelist2 = []
for x in incomelist:
    x1 = str(x)
    x1 = x1.replace('Between $150,000 and $199,000','More than $150,000')
    x1 = x1.replace('More than $200,000','More than $150,000')
    x1 = x1.replace('high_iii','More than $150,000')
    x1 = x1.replace('high_ii','Between $125,000 and $149,999')
    x1 = x1.replace('high_i','Between $100,000 and $124,999')
    x1 = x1.replace('middle_ii','Between $75,000 and $99,999')
    x1 = x1.replace('middle_i','Between $50,000 and $74,999')
    x1 = x1.replace('lower_ii','Between $25,000 and $49,999')
    x1 = x1.replace('lower_i','Less than $25,000')
    
    x1 = x1.replace('prefer_not_to_say','Prefer not to say')
    incomelist2.append(x1)
data['income_group'] = incomelist2
data['income_group'] = data['income_group'].str.replace('$','$\$$')

## Ethnicity column cleaning

In [26]:
data['Ethnicity'] = data['What is your ethnicity?'].str.replace('White','Caucasian')
data['Ethnicity'] = data['Ethnicity'].str.replace('Hispanic/latinx','Hispanic')
data['Ethnicity'] = data['Ethnicity'].str.replace('Latino','Hispanic')
data['Ethnicity'] = data['Ethnicity'].str.replace('Hispanic/latinx','Hispanic')
data['Ethnicity'] = data['Ethnicity'].str.replace('Black/african american','Black')
data['Ethnicity'] = data['Ethnicity'].str.replace('Caucasian, asian/asian american','Multiracial')
data['Ethnicity'] = data['Ethnicity'].str.replace('Caucasian, hispanic/latinx','Multiracial')
data['Ethnicity'] = data['Ethnicity'].str.replace('Asian/asian american','Asian')
data['Ethnicity'] = data['Ethnicity'].str.replace('Caucasian, armenian','Multiracial')
data['Ethnicity'] = data['Ethnicity'].str.replace('Asian, other','Asian')
data['Ethnicity'] = data['Ethnicity'].str.replace('Caucasian, jewish','Caucasian')
data['Ethnicity'] = data['Ethnicity'].str.replace('Caucasian, asian','Multiracial')
data['Ethnicity'] = data['Ethnicity'].str.replace('Multiracial/asian american','Multiracial')
data['Ethnicity'] = data['Ethnicity'].str.replace('Arab','Asian')
data['Ethnicity'] = data['Ethnicity'].str.replace('Asian, pacific islander','Asian')
data['Ethnicity'] = data['Ethnicity'].str.replace('Prefer_not_to_say','Prefer not to say')